# Multi-Agent Customer Service System with A2A and MCP

## System Architecture

This notebook implements a production-ready multi-agent customer service system with:

### Agents:
1. **Router Agent (Orchestrator)**: Routes queries and coordinates multi-agent responses
2. **Customer Data Agent (Specialist)**: Manages customer data via MCP protocol
3. **Support Agent (Specialist)**: Handles support queries and escalations

### Key Features:
- Agent-to-Agent (A2A) Communication
- Model Context Protocol (MCP) for data access
- Intelligent query routing
- Multi-agent coordination
- State management with LangGraph

## 1. Installation and Setup

In [ ]:
# Install required libraries
!pip install -q -U langgraph langchain-openai langchain-core pydantic typing-extensions

In [ ]:
# Import dependencies
import os
from typing import Literal, TypedDict, Annotated, Optional, List, Dict, Any
from datetime import datetime
from enum import Enum
import json

from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pydantic import BaseModel, Field

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("✓ Dependencies loaded successfully")

## 2. MCP Server Implementation

Model Context Protocol (MCP) provides standardized access to customer data.
In production, this would connect to a real database or API.

In [ ]:
# MCP Data Models
class CustomerRecord(BaseModel):
    """Customer data schema"""
    customer_id: str
    name: str
    email: str
    phone: str
    tier: Literal["basic", "premium", "enterprise"]
    account_status: Literal["active", "suspended", "closed"]
    join_date: str
    total_orders: int
    lifetime_value: float
    support_tickets: List[Dict[str, Any]] = Field(default_factory=list)

class MCPServer:
    """Simulated MCP Server for Customer Data"""
    
    def __init__(self):
        # Simulated customer database
        self.customers = {
            "CUST001": CustomerRecord(
                customer_id="CUST001",
                name="Alice Johnson",
                email="alice.johnson@email.com",
                phone="+1-555-0101",
                tier="premium",
                account_status="active",
                join_date="2023-01-15",
                total_orders=47,
                lifetime_value=12450.00,
                support_tickets=[
                    {"id": "TK001", "status": "resolved", "issue": "Shipping delay"},
                    {"id": "TK015", "status": "open", "issue": "Product defect"}
                ]
            ),
            "CUST002": CustomerRecord(
                customer_id="CUST002",
                name="Bob Martinez",
                email="bob.m@company.com",
                phone="+1-555-0202",
                tier="enterprise",
                account_status="active",
                join_date="2022-06-10",
                total_orders=203,
                lifetime_value=87600.00,
                support_tickets=[
                    {"id": "TK042", "status": "resolved", "issue": "Billing inquiry"}
                ]
            ),
            "CUST003": CustomerRecord(
                customer_id="CUST003",
                name="Carol White",
                email="carol.white@mail.com",
                phone="+1-555-0303",
                tier="basic",
                account_status="active",
                join_date="2024-02-20",
                total_orders=3,
                lifetime_value=287.50,
                support_tickets=[]
            )
        }
    
    def get_customer(self, customer_id: str) -> Optional[CustomerRecord]:
        """Retrieve customer by ID via MCP protocol"""
        return self.customers.get(customer_id)
    
    def search_customer(self, email: str = None, phone: str = None) -> Optional[CustomerRecord]:
        """Search customer by email or phone"""
        for customer in self.customers.values():
            if email and customer.email == email:
                return customer
            if phone and customer.phone == phone:
                return customer
        return None
    
    def update_customer(self, customer_id: str, updates: Dict[str, Any]) -> bool:
        """Update customer record"""
        if customer_id not in self.customers:
            return False
        
        customer = self.customers[customer_id]
        for key, value in updates.items():
            if hasattr(customer, key):
                setattr(customer, key, value)
        return True
    
    def add_support_ticket(self, customer_id: str, ticket: Dict[str, Any]) -> bool:
        """Add support ticket to customer record"""
        if customer_id not in self.customers:
            return False
        
        self.customers[customer_id].support_tickets.append(ticket)
        return True

# Initialize MCP Server
mcp_server = MCPServer()
print("✓ MCP Server initialized with", len(mcp_server.customers), "customers")

## 3. Agent State Schema

Defines the shared state that flows through the multi-agent system.

In [ ]:
class QueryIntent(str, Enum):
    """Types of customer queries"""
    ACCOUNT_INFO = "account_info"
    ORDER_STATUS = "order_status"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    GENERAL_INQUIRY = "general_inquiry"
    COMPLAINT = "complaint"

class AgentState(TypedDict):
    """Shared state across all agents"""
    # Input
    query: str
    customer_identifier: Optional[str]  # ID, email, or phone
    
    # Router decisions
    intent: Optional[str]
    requires_data_agent: bool
    requires_support_agent: bool
    
    # Customer data (from Data Agent)
    customer_data: Optional[Dict[str, Any]]
    
    # Agent responses
    data_agent_response: Optional[str]
    support_agent_response: Optional[str]
    
    # Final output
    response: str
    escalation_needed: bool
    
    # Workflow tracking
    current_phase: str
    agents_consulted: List[str]

print("✓ State schema defined")

## 4. Router Agent (Orchestrator)

The Router Agent analyzes queries and coordinates the workflow between specialist agents.

In [ ]:
# Initialize LLM for Router
router_llm = ChatOpenAI(model="gpt-4o", temperature=0)

ROUTER_SYSTEM_PROMPT = """You are the Router Agent in a customer service system.

Your responsibilities:
1. Analyze customer queries to determine intent
2. Identify which specialist agents are needed:
   - Customer Data Agent: For account info, order history, customer details
   - Support Agent: For technical issues, complaints, general support
3. Extract customer identifiers (ID, email, phone) if present
4. Coordinate responses from multiple agents when needed

Intent categories:
- account_info: Account status, profile updates
- order_status: Order tracking, delivery questions
- technical_support: Product issues, troubleshooting
- billing: Payment, invoices, refunds
- general_inquiry: Product info, policies
- complaint: Service complaints, escalations

Return your analysis in JSON format:
{
  "intent": "<intent_category>",
  "requires_data_agent": true/false,
  "requires_support_agent": true/false,
  "customer_identifier": "<ID/email/phone or null>"
}
"""

def router_agent(state: AgentState) -> AgentState:
    """Router Agent: Analyze query and determine routing"""
    print("\n🔀 ROUTER AGENT")
    print(f"   Query: {state['query']}")
    
    # Analyze query using LLM
    response = router_llm.invoke([
        SystemMessage(content=ROUTER_SYSTEM_PROMPT),
        HumanMessage(content=f"Analyze this customer query: {state['query']}")
    ])
    
    # Parse LLM response
    try:
        analysis = json.loads(response.content)
    except:
        # Fallback if JSON parsing fails
        analysis = {
            "intent": "general_inquiry",
            "requires_data_agent": False,
            "requires_support_agent": True,
            "customer_identifier": state.get('customer_identifier')
        }
    
    print(f"   Intent: {analysis['intent']}")
    print(f"   Data Agent needed: {analysis['requires_data_agent']}")
    print(f"   Support Agent needed: {analysis['requires_support_agent']}")
    
    return {
        **state,
        "intent": analysis["intent"],
        "requires_data_agent": analysis["requires_data_agent"],
        "requires_support_agent": analysis["requires_support_agent"],
        "customer_identifier": analysis.get("customer_identifier") or state.get('customer_identifier'),
        "current_phase": "routing_complete",
        "agents_consulted": ["router"]
    }

print("✓ Router Agent defined")

## 5. Customer Data Agent (Specialist)

The Data Agent interfaces with the MCP server to retrieve and update customer information.

In [ ]:
# Initialize LLM for Data Agent
data_agent_llm = ChatOpenAI(model="gpt-4o", temperature=0)

DATA_AGENT_SYSTEM_PROMPT = """You are the Customer Data Agent.

Your responsibilities:
1. Access customer data via MCP protocol
2. Retrieve customer information (profile, orders, history)
3. Update customer records when authorized
4. Validate data integrity
5. Provide context to other agents

You have access to:
- Customer profiles
- Order history
- Support ticket history
- Account status

Always protect sensitive data and follow privacy guidelines.
"""

def customer_data_agent(state: AgentState) -> AgentState:
    """Customer Data Agent: Retrieve and manage customer data via MCP"""
    print("\n💾 CUSTOMER DATA AGENT")
    
    customer_identifier = state.get('customer_identifier')
    
    if not customer_identifier:
        print("   ⚠️  No customer identifier provided")
        return {
            **state,
            "data_agent_response": "Unable to retrieve customer data: No identifier provided",
            "customer_data": None,
            "agents_consulted": state['agents_consulted'] + ["data_agent"]
        }
    
    # Try to retrieve customer data via MCP
    customer = None
    
    # Check if identifier is customer ID
    if customer_identifier.startswith("CUST"):
        customer = mcp_server.get_customer(customer_identifier)
    # Check if identifier is email
    elif "@" in customer_identifier:
        customer = mcp_server.search_customer(email=customer_identifier)
    # Check if identifier is phone
    else:
        customer = mcp_server.search_customer(phone=customer_identifier)
    
    if not customer:
        print(f"   ❌ Customer not found: {customer_identifier}")
        return {
            **state,
            "data_agent_response": f"Customer not found with identifier: {customer_identifier}",
            "customer_data": None,
            "agents_consulted": state['agents_consulted'] + ["data_agent"]
        }
    
    # Convert customer data to dict
    customer_dict = customer.dict()
    
    print(f"   ✓ Found customer: {customer.name}")
    print(f"   Tier: {customer.tier} | Status: {customer.account_status}")
    print(f"   Orders: {customer.total_orders} | LTV: ${customer.lifetime_value:,.2f}")
    
    # Generate contextual response using LLM
    response = data_agent_llm.invoke([
        SystemMessage(content=DATA_AGENT_SYSTEM_PROMPT),
        HumanMessage(content=f"""Customer Query: {state['query']}
        
Customer Data:
{json.dumps(customer_dict, indent=2)}

Provide a summary of relevant customer information for this query.""")
    ])
    
    return {
        **state,
        "customer_data": customer_dict,
        "data_agent_response": response.content,
        "current_phase": "data_retrieved",
        "agents_consulted": state['agents_consulted'] + ["data_agent"]
    }

print("✓ Customer Data Agent defined")

## 6. Support Agent (Specialist)

The Support Agent handles customer inquiries, provides solutions, and can escalate complex issues.

In [ ]:
# Initialize LLM for Support Agent
support_agent_llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

SUPPORT_AGENT_SYSTEM_PROMPT = """You are the Support Agent in a customer service system.

Your responsibilities:
1. Handle customer support queries professionally
2. Provide solutions and recommendations
3. Use customer context from Data Agent when available
4. Escalate complex or sensitive issues
5. Document support interactions

Escalation criteria:
- Legal or compliance issues
- High-value customer complaints (enterprise tier)
- Technical issues requiring engineering
- Requests for refunds over $500

Always be:
- Professional and empathetic
- Clear and concise
- Solution-oriented
- Privacy-conscious
"""

def support_agent(state: AgentState) -> AgentState:
    """Support Agent: Handle customer support queries"""
    print("\n🎧 SUPPORT AGENT")
    
    # Build context from customer data if available
    context = ""
    if state.get('customer_data'):
        customer = state['customer_data']
        context = f"""\n\nCustomer Context:
- Name: {customer['name']}
- Tier: {customer['tier']}
- Account Status: {customer['account_status']}
- Total Orders: {customer['total_orders']}
- Open Tickets: {len([t for t in customer.get('support_tickets', []) if t['status'] == 'open'])}
"""
        print(f"   ✓ Using customer context for {customer['name']}")
    
    # Generate support response
    response = support_agent_llm.invoke([
        SystemMessage(content=SUPPORT_AGENT_SYSTEM_PROMPT),
        HumanMessage(content=f"""Customer Query: {state['query']}
        
Intent: {state.get('intent', 'unknown')}{context}

Provide a helpful response. If this requires escalation, start your response with "[ESCALATION NEEDED]""")
    ])
    
    # Check if escalation is needed
    escalation_needed = response.content.startswith("[ESCALATION NEEDED]")
    
    if escalation_needed:
        print("   ⚠️  Escalation required")
    else:
        print("   ✓ Response generated")
    
    return {
        **state,
        "support_agent_response": response.content,
        "escalation_needed": escalation_needed,
        "current_phase": "support_complete",
        "agents_consulted": state['agents_consulted'] + ["support_agent"]
    }

print("✓ Support Agent defined")

## 7. Response Synthesizer

Combines responses from multiple agents into a coherent final response.

In [ ]:
# Initialize LLM for synthesis
synthesizer_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

def response_synthesizer(state: AgentState) -> AgentState:
    """Synthesize final response from all agent outputs"""
    print("\n🔄 SYNTHESIZING RESPONSE")
    
    # Collect all agent responses
    responses = []
    
    if state.get('data_agent_response'):
        responses.append(f"Customer Data:\n{state['data_agent_response']}")
    
    if state.get('support_agent_response'):
        responses.append(f"Support Response:\n{state['support_agent_response']}")
    
    if not responses:
        final_response = "I apologize, but I was unable to process your request. Please try again or contact support."
    elif len(responses) == 1:
        # Single agent response
        final_response = responses[0].split(':\n', 1)[1]
    else:
        # Multi-agent response - synthesize
        synthesis_prompt = f"""Combine these agent responses into a single, coherent customer service response:

{chr(10).join(responses)}

Create a professional, helpful response that:
1. Addresses the customer's query directly
2. Incorporates relevant information from all agents
3. Maintains a friendly, professional tone
4. Is concise and well-structured
"""
        
        response = synthesizer_llm.invoke([
            HumanMessage(content=synthesis_prompt)
        ])
        
        final_response = response.content
    
    print(f"   ✓ Final response synthesized")
    print(f"   Agents consulted: {', '.join(state['agents_consulted'])}")
    
    return {
        **state,
        "response": final_response,
        "current_phase": "complete"
    }

print("✓ Response Synthesizer defined")

## 8. Build the Multi-Agent Workflow

Connect all agents into a coordinated LangGraph workflow with conditional routing.

In [ ]:
# Routing logic functions
def route_after_router(state: AgentState) -> List[str]:
    """Determine which agents to call after routing"""
    agents_to_call = []
    
    if state.get('requires_data_agent'):
        agents_to_call.append("data_agent")
    
    if state.get('requires_support_agent'):
        agents_to_call.append("support_agent")
    
    # If no specific agent needed, default to support
    if not agents_to_call:
        agents_to_call.append("support_agent")
    
    return agents_to_call

def should_call_support_after_data(state: AgentState) -> str:
    """Decide if support agent needed after data agent"""
    if state.get('requires_support_agent'):
        return "support_agent"
    return "synthesizer"

# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("router", router_agent)
workflow.add_node("data_agent", customer_data_agent)
workflow.add_node("support_agent", support_agent)
workflow.add_node("synthesizer", response_synthesizer)

# Add edges
workflow.add_edge(START, "router")

# Conditional routing from router
workflow.add_conditional_edges(
    "router",
    route_after_router,
    {
        "data_agent": "data_agent",
        "support_agent": "support_agent"
    }
)

# Conditional routing from data agent
workflow.add_conditional_edges(
    "data_agent",
    should_call_support_after_data,
    {
        "support_agent": "support_agent",
        "synthesizer": "synthesizer"
    }
)

# Support agent always goes to synthesizer
workflow.add_edge("support_agent", "synthesizer")

# Synthesizer is the end
workflow.add_edge("synthesizer", END)

# Compile the graph
customer_service_system = workflow.compile()

print("✓ Multi-Agent Workflow compiled successfully")
print("\n📊 Workflow Structure:")
print("   START → Router → [Data Agent] → [Support Agent] → Synthesizer → END")

## 9. Test Cases

Demonstrate the system with various customer service scenarios.

### Test Case 1: Account Information Query (Data Agent Only)

In [ ]:
print("=" * 80)
print("TEST CASE 1: Account Information Query")
print("=" * 80)

result = customer_service_system.invoke({
    "query": "Can you show me my account details and order history?",
    "customer_identifier": "CUST001",
    "current_phase": "initial",
    "agents_consulted": [],
    "escalation_needed": False,
    "requires_data_agent": False,
    "requires_support_agent": False
})

print("\n" + "=" * 80)
print("FINAL RESPONSE")
print("=" * 80)
print(result['response'])
print("\nEscalation needed:", result['escalation_needed'])
print("Agents consulted:", ', '.join(result['agents_consulted']))

### Test Case 2: Technical Support Query (Support Agent with Customer Context)

In [ ]:
print("=" * 80)
print("TEST CASE 2: Technical Support with Customer Context")
print("=" * 80)

result = customer_service_system.invoke({
    "query": "The product I received is defective and doesn't work as advertised. I need a replacement or refund.",
    "customer_identifier": "alice.johnson@email.com",
    "current_phase": "initial",
    "agents_consulted": [],
    "escalation_needed": False,
    "requires_data_agent": False,
    "requires_support_agent": False
})

print("\n" + "=" * 80)
print("FINAL RESPONSE")
print("=" * 80)
print(result['response'])
print("\nEscalation needed:", result['escalation_needed'])
print("Agents consulted:", ', '.join(result['agents_consulted']))

### Test Case 3: Enterprise Customer Issue (Escalation)

In [ ]:
print("=" * 80)
print("TEST CASE 3: Enterprise Customer - Potential Escalation")
print("=" * 80)

result = customer_service_system.invoke({
    "query": "We've been experiencing critical system outages affecting our business operations. This is unacceptable for an enterprise client. We need immediate resolution and compensation.",
    "customer_identifier": "CUST002",
    "current_phase": "initial",
    "agents_consulted": [],
    "escalation_needed": False,
    "requires_data_agent": False,
    "requires_support_agent": False
})

print("\n" + "=" * 80)
print("FINAL RESPONSE")
print("=" * 80)
print(result['response'])
print("\nEscalation needed:", result['escalation_needed'])
print("Agents consulted:", ', '.join(result['agents_consulted']))

### Test Case 4: General Inquiry (No Customer Context)

In [ ]:
print("=" * 80)
print("TEST CASE 4: General Inquiry - No Customer ID")
print("=" * 80)

result = customer_service_system.invoke({
    "query": "What are your business hours and shipping policies?",
    "customer_identifier": None,
    "current_phase": "initial",
    "agents_consulted": [],
    "escalation_needed": False,
    "requires_data_agent": False,
    "requires_support_agent": False
})

print("\n" + "=" * 80)
print("FINAL RESPONSE")
print("=" * 80)
print(result['response'])
print("\nEscalation needed:", result['escalation_needed'])
print("Agents consulted:", ', '.join(result['agents_consulted']))

### Test Case 5: Multi-Agent Coordination (Billing with Account Check)

In [ ]:
print("=" * 80)
print("TEST CASE 5: Complex Query - Multi-Agent Coordination")
print("=" * 80)

result = customer_service_system.invoke({
    "query": "I was charged twice for my last order. Can you check my account and help me get a refund?",
    "customer_identifier": "+1-555-0303",
    "current_phase": "initial",
    "agents_consulted": [],
    "escalation_needed": False,
    "requires_data_agent": False,
    "requires_support_agent": False
})

print("\n" + "=" * 80)
print("FINAL RESPONSE")
print("=" * 80)
print(result['response'])
print("\nEscalation needed:", result['escalation_needed'])
print("Agents consulted:", ', '.join(result['agents_consulted']))

## 10. Interactive Demo

Try your own queries with the customer service system.

In [ ]:
def process_customer_query(query: str, customer_id: str = None):
    """Process a customer service query through the multi-agent system"""
    print("\n" + "="*80)
    print("PROCESSING CUSTOMER QUERY")
    print("="*80)
    print(f"Query: {query}")
    if customer_id:
        print(f"Customer: {customer_id}")
    print("="*80)
    
    result = customer_service_system.invoke({
        "query": query,
        "customer_identifier": customer_id,
        "current_phase": "initial",
        "agents_consulted": [],
        "escalation_needed": False,
        "requires_data_agent": False,
        "requires_support_agent": False
    })
    
    print("\n" + "="*80)
    print("RESPONSE")
    print("="*80)
    print(result['response'])
    print("\n" + "-"*80)
    print(f"Intent: {result.get('intent', 'N/A')}")
    print(f"Agents: {', '.join(result['agents_consulted'])}")
    print(f"Escalation: {'Yes' if result['escalation_needed'] else 'No'}")
    print("="*80)
    
    return result

# Example usage:
# process_customer_query(
#     "I want to upgrade my account to premium tier",
#     "CUST003"
# )

## 11. System Analytics

View metrics and performance of the multi-agent system.

In [ ]:
def display_system_status():
    """Display current system status and statistics"""
    print("\n" + "="*80)
    print("MULTI-AGENT CUSTOMER SERVICE SYSTEM - STATUS")
    print("="*80)
    
    print("\n📊 SYSTEM COMPONENTS:")
    print("   ✓ Router Agent (Orchestrator)")
    print("   ✓ Customer Data Agent (MCP-enabled)")
    print("   ✓ Support Agent (Specialist)")
    print("   ✓ Response Synthesizer")
    
    print("\n💾 MCP SERVER STATUS:")
    print(f"   Total Customers: {len(mcp_server.customers)}")
    
    for cust_id, customer in mcp_server.customers.items():
        print(f"\n   {cust_id}: {customer.name}")
        print(f"      Tier: {customer.tier} | Status: {customer.account_status}")
        print(f"      Orders: {customer.total_orders} | LTV: ${customer.lifetime_value:,.2f}")
        print(f"      Open Tickets: {len([t for t in customer.support_tickets if t['status'] == 'open'])}")
    
    print("\n🔧 CAPABILITIES:")
    print("   • Intent classification (6 categories)")
    print("   • Dynamic agent routing")
    print("   • Multi-agent coordination")
    print("   • Customer data retrieval via MCP")
    print("   • Automatic escalation detection")
    print("   • Context-aware responses")
    
    print("\n" + "="*80)

display_system_status()

## 12. Architecture Summary

### System Flow:

```
1. CUSTOMER QUERY RECEIVED
   ↓
2. ROUTER AGENT
   - Analyzes intent
   - Extracts customer identifier
   - Determines required agents
   ↓
3. SPECIALIST AGENTS (Parallel/Sequential)
   
   3a. CUSTOMER DATA AGENT (if needed)
       - Connects to MCP server
       - Retrieves customer data
       - Validates information
   
   3b. SUPPORT AGENT (if needed)
       - Uses customer context
       - Generates solution
       - Checks escalation criteria
   ↓
4. RESPONSE SYNTHESIZER
   - Combines agent outputs
   - Creates coherent response
   - Formats final answer
   ↓
5. RESPONSE DELIVERED
```

### Key Technologies:
- **LangGraph**: Multi-agent orchestration and state management
- **OpenAI GPT-4**: Natural language understanding and generation
- **MCP (Model Context Protocol)**: Standardized data access
- **Pydantic**: Data validation and schema enforcement

### Agent Communication (A2A):
- Agents communicate through shared state
- Router coordinates information flow
- Support Agent can request data from Data Agent
- All interactions logged for audit trail

### Production Considerations:
1. **Security**: Implement authentication, authorization, encryption
2. **Scalability**: Add caching, load balancing, async processing
3. **Monitoring**: Add logging, metrics, alerting
4. **Error Handling**: Implement retry logic, fallbacks, circuit breakers
5. **Data Privacy**: PII protection, GDPR compliance, data retention policies